In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        print("Found a GPU with ID: ",gpu)
else:
    print("Failed to detect a GPU.")

Failed to detect a GPU.


In [4]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pprint
import torch.nn as nn

import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, Conv2d,MaxPool1d, MaxPool2d, Module, Softmax, BatchNorm1d, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

# torchvision for pre-trained models
from torchvision import models

import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision import transforms


In [5]:
master_csv = pd.read_csv("MASTER_CSV_4.csv")
master_csv.head()
master_csv.shape


(12660, 4)

In [6]:
cwd = os.getcwd() 
folder_path = os.path.join(cwd,"session_1")

# define custom transform function
transform_tensor = transforms.Compose([transforms.ToTensor()])


eeg_data = []
for filename in tqdm(master_csv['filename']):
    file_path = os.path.join(folder_path, filename)
    file = pd.read_csv(file_path)
    
    # Drop rows below row 6002
    #file = file.iloc[:6002]
    
    file = file.astype('float32')
    #eeg_values = file.drop(columns=['time']).values  # Assuming 'time' column is not needed
    
    # Calculate mean and std
    mean, std = file.mean(), file.std()
    
    # Normalize EEG data
    normalized_data = (file - mean) / std
    
    eeg_data.append(normalized_data)

# Convert the list of normalized arrays to a NumPy array
eeg_numpy = np.array(eeg_data)
print(eeg_numpy.shape)



100%|████████████████████████████████████████████████████████████████████████████| 12660/12660 [14:14<00:00, 14.82it/s]


(12660, 1001, 62)


In [7]:
#eeg_data

In [8]:
eeg_numpy

array([[[ 1.1158739 ,  1.236367  ,  1.2142688 , ...,  1.4467032 ,
          1.6951935 ,  1.7026098 ],
        [ 1.6328441 ,  1.443384  ,  1.3650314 , ...,  1.7051029 ,
          1.8625292 ,  1.8763801 ],
        [ 1.2299114 ,  1.7298721 ,  1.1660248 , ...,  1.3870724 ,
          1.5190508 ,  1.5187662 ],
        ...,
        [-0.9697455 , -1.283159  , -1.5356385 , ..., -1.0816393 ,
         -0.64247245, -0.60677016],
        [-1.1737459 , -1.7648908 , -1.7635913 , ..., -1.114436  ,
         -0.46003878, -0.43174088],
        [-0.9127267 , -1.3714602 , -1.0230461 , ..., -1.2674884 ,
         -0.83622926, -0.8774991 ]],

       [[ 2.528494  ,  2.4359193 ,  2.4094703 , ...,  0.3816055 ,
          0.35974196,  0.43337438],
        [ 3.455898  ,  3.6334217 ,  3.503844  , ..., -0.30513126,
         -0.35826373, -0.38696492],
        [ 4.0914726 ,  4.528642  ,  4.5213394 , ...,  0.5561996 ,
          0.42966947,  0.41957662],
        ...,
        [-0.98761904, -0.9482108 , -0.93605655, ...,  

In [9]:
eeg_numpy.shape # 360, 400, 62

(12660, 1001, 62)

In [10]:
# defining the target
label_emotion = master_csv['emotion'].values
print(label_emotion[:48])  # Display the first 10 elements

[2 2 3 3 0 3 0 1 0 2 1 3 1 3 1 1 0 2 2 2 2 3 1 2 0 3 3 3 3 1 2 1 1 0 3 3 0
 0 1 2 2 3 0 2 1 3 3 1]


In [11]:
# create validation set
train_x, val_x, train_y, val_y = train_test_split(eeg_numpy, label_emotion, test_size = 0.25, random_state = 13, stratify=label_emotion)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((9495, 1001, 62), (9495,)), ((3165, 1001, 62), (3165,)))

In [12]:
# converting training images into torch format
train_x = train_x.reshape(9495, 1, 1001, 62)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y).long()

# shape of training data
train_x.shape, train_y.shape

# converting validation images into torch format
val_x = val_x.reshape(3165, 1, 1001, 62)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y).long()

# shape of validation data
val_x.shape, val_y.shape

(torch.Size([3165, 1, 1001, 62]), torch.Size([3165]))

In [13]:
class ConvNetGray(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Adding pooling layer here to reduce spatial dimensions
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Final pooling to reduce size further if needed
        )
        self.fc = nn.Sequential(
            nn.Linear(128 * 125 * 7, 256),  # Adjusted size_after_conv to match output size from conv layers
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.Linear(64, 4)
        )

    def forward(self, x):
        out = self.conv_layers(x)
#         print(f"Output shape after convolutions: {out.shape}")  # This will print the output shape
        out = out.view(out.size(0), -1)  # Flatten the output
        out = self.fc(out)
        return out

# nn.Linear(512 * 31 * 1, 256),

In [19]:
import torch
print(torch.cuda.is_available())

False


In [18]:
# defining the model
model = ConvNetGray()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0001)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = model.to(device)
criterion = criterion.to(device)

print(model)

Using device: cpu
ConvNetGray(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=112000, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_fea

In [15]:
%%time
# batch size of the model
batch_size = 128

# Define parameters
n_epochs = 10  # Define the maximum number of epochs
early_stop_patience = 10  # Define patience for early stopping
min_loss = float('inf')  # Initialize minimum loss
best_epoch = 0  # Initialize the epoch with the best performance
no_improvement_count = 0  # Initialize the count for early stopping

train_losses = []  # List to store training losses

# number of epochs to train the model
#n_epochs = 15

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
        
    permutation = torch.randperm(train_x.size()[0])
    #permutation = torch.randperm(train_x.size()[0])

    training_loss = []
    for i in tqdm(range(0,train_x.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        #if torch.cuda.is_available():
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        # in case you wanted a semi-full example
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        train_losses.append(training_loss)

    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

    # Check for early stopping
    if training_loss < min_loss:
        min_loss = training_loss
        best_epoch = epoch
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stop_patience:
        print(f'Early stopping at epoch {epoch} with minimum loss {min_loss}')
        break

100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [15:43<00:00, 12.57s/it]

epoch: 	 1 	 training loss: 	 1.7251576296488444
CPU times: total: 1h 23min 41s
Wall time: 15min 43s


In [16]:
%%time
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in tqdm(range(0,train_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i],prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))


100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [04:08<00:00,  3.31s/it]

training accuracy: 	 0.34405797101449276
CPU times: total: 21min 48s
Wall time: 4min 8s


In [17]:
%%time
# prediction for validation set
prediction_val = []
target_val = []
permutation = torch.randperm(val_x.size()[0])
for i in tqdm(range(0,val_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = val_x[indices], val_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction_val.append(predictions)
    target_val.append(batch_y)
    
# validation accuracy
accuracy_val = []
for i in range(len(prediction_val)):
    accuracy_val.append(accuracy_score(target_val[i],prediction_val[i]))
    
print('validation accuracy: \t', np.average(accuracy_val))

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [01:25<00:00,  3.43s/it]

validation accuracy: 	 0.3215356182795699
CPU times: total: 7min 32s
Wall time: 1min 25s
